In [6]:
!pip install pyphen
!pip install nltk

In [1]:
import os
import pandas as pd
import codecs
import re
import xml.etree.ElementTree as ET
import numpy as np
import shutil
from shutil import copyfile
import emoji
import re
import operator
from collections import Counter, OrderedDict, defaultdict

In [5]:
!python -m spacy download en_core_web_md

  Stored in directory: C:\Users\Vanda\AppData\Local\Temp\pip-ephem-wheel-cache-fz9f5liv\wheels\db\5d\d0\ccdad6b01f9695b4a33793158530b3228223ee31463441663d
Successfully built en-core-web-md

    Error: Couldn't link model to 'en_core_web_md'
    Creating a symlink in spacy/data failed. Make sure you have the required
    permissions and try re-running the command as admin, or use a
    virtualenv. You can still import the model as a module and call its
    load() method, or create the symlink manually.

    C:\Users\Vanda\Anaconda3\lib\site-packages\en_core_web_md -->
    C:\Users\Vanda\Anaconda3\lib\site-packages\spacy\data\en_core_web_md


    Download successful
    Creating a shortcut link for 'en' didn't work (maybe you don't have
    admin permissions?), but you can still load the model via its full
    package name:

    nlp = spacy.load('en_core_web_md')



In [2]:
import pyphen
PYPHEN_DIC = pyphen.Pyphen(lang='en')
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
import spacy
spacy_nlp = spacy.load('en_core_web_md')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Character flooding
eg looooove, floooood
Find words that have consecutive characters appearing in them more than 2 times

In [4]:
def CharacterFlooding(tweet):
    tknzr = TweetTokenizer()
    tweet_words = tknzr.tokenize(tweet)
    floodings = 0
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    for word in tweet_words:
        if not regexpURL.search(word) and len(re.findall(r'(\w)\1{2,}',word)) > 0: 
            floodings += 1
    return floodings

### Counting Capital Letters

In [5]:
def NumberOfCapitalLetters(tweet):
    now, words = NumberOfWords(tweet)
    nocl = 0
    for word in words:
        nocl += sum(1 for c in word if c.isupper())
    avg_nocl = nocl / now
    return nocl, avg_nocl

### Counting Emoticons

In [6]:
def NumberOfEmoticons(tweet):
    emojis = [i for i in tweet if i in emoji.UNICODE_EMOJI]
    return len(emojis)

### Exploring Topics

In [7]:
import pickle
SEMCAT_word_concept_dict = pickle.load(open('C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/kb/SEMCAT2018_word_concept_dict.p', 'rb'), encoding='utf-8')
semcor_word_concept_dict = pickle.load(open('C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/kb/semcor_noun_verb.supersenses.en_word_concept_dict.p', 'rb'), encoding='utf-8')
SEMCAT_concepts = set([item for sublist in SEMCAT_word_concept_dict.values() for item in sublist])
semcor_concepts = set([item for sublist in semcor_word_concept_dict.values() for item in sublist])
# merge
wordTopicsDict = SEMCAT_word_concept_dict
for word, concept_set in semcor_word_concept_dict.items():
    wordTopicsDict[word] = wordTopicsDict[word].union(concept_set)
topics = SEMCAT_concepts.union(semcor_concepts)
topicNames = sorted(list(topics), key=lambda t: t)
print(wordTopicsDict['car'], SEMCAT_word_concept_dict['car'], semcor_word_concept_dict['car'])
print("Number of topics: ", len(topics))
print(topicNames)

{'carparts', 'vacation', 'driving', 'artifact', 'car'} {'carparts', 'vacation', 'driving', 'artifact', 'car'} {'artifact'}
Number of topics:  133
['Tops', 'act', 'adjectives_for_people', 'animal', 'april_fool', 'art', 'artifact', 'astronomy', 'attribute', 'baseball', 'bathroom', 'beach', 'big', 'biomes', 'birds', 'birthday', 'boat', 'bodies_of_water', 'body', 'buildings', 'camping', 'car', 'carnival', 'carparts', 'castle', 'cats', 'change', 'christmas', 'circus', 'clothes', 'cognition', 'colors', 'communication', 'competition', 'computer', 'constitution', 'consumption', 'contact', 'container', 'cooking', 'cooking_tools', 'country', 'creation', 'dance', 'dentist', 'desserts', 'doctor', 'dogs', 'driving', 'election', 'emotion', 'emotions', 'energy', 'event', 'fall', 'family', 'farm', 'feeling', 'fish', 'flowers', 'food', 'foodweb', 'fruit', 'furniture', 'geography', 'grammar', 'group', 'happiness', 'happy', 'house', 'housing', 'insect', 'jobs', 'kitchen', 'land_forms', 'languages', 'lead

In [8]:
def Topics(tweet):
    topicsFrequency = {t:0 for t in topics}
    tweet = spacy_nlp(tweet)
    lemmas = [token.lemma_ for token in tweet]
    for token in tweet:
        lemma = token.lemma_
        topicSet = wordTopicsDict.get(lemma, set())
        for t in topicSet:
            topicsFrequency[t] += 1
    topicsFrequency = OrderedDict(sorted(topicsFrequency.items(), key=lambda e: e[0]))
    topicsValues = list(topicsFrequency.values())
    return topicsValues

### Counting URLs

In [9]:
def NumberOfURLs(tweet):
    tknzr = TweetTokenizer()
    tweet_words = tknzr.tokenize(tweet)
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    urlNumber = 0
    for word in tweet_words:
        if regexpURL.search(word): 
            urlNumber += 1
    return urlNumber

### Counting Repeated Words
Compute:
- number of tokens repeated more (or equal) than *k* times 
- maximum number of repetition (of a single token) (>=*k*)

For example (k=3) in the tweet: *"Hairy cats like other cats that are not hairy. However, hairy dogs like cats that are not hairy."*
- the tokens repeated more (or equal) than 3 times are *hairy* and *cats*, so the number of tokens repeated more than 3 times is 2
- the token *hairy* is repeated most of the time: 4 times

In [10]:
def wordsRepeated(tweet, k=3):

    frequency = defaultdict(int)
    tweet = spacy_nlp(tweet)
    for token in tweet:
        if token.lemma_ not in spacy_stopwords:
            frequency[token.lemma_] += 1
    maxAppearance = 0
    numberOfTokensRepeated = 0
    if len(frequency.items()) > 0:
        maxAppearance = max(frequency.items(), key=operator.itemgetter(1))[1]
        numberOfTokensRepeated = sum(1 for (lemma, freq) in frequency.items() if freq >= k)
    return maxAppearance, numberOfTokensRepeated

### Number of POS tags

In [11]:
all_pos_tags = ["NO_TAG", "ADJ", "ADP", "ADV","AUX", "CONJ","CCONJ","DET",
                      "INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM",
                      "VERB","X","EOL","SPACE"]

In [12]:
def POSTags(tweet):
    tweet = spacy_nlp(tweet)
#     pos_list = []
    c = Counter()
    c.update({x:0 for x in all_pos_tags})
    pos_list = [token.pos_ for token in tweet]
    assert len(set(pos_list).difference(set(all_pos_tags))) == 0
    c.update(pos_list)
    c = OrderedDict(sorted(c.items(), key=lambda e: e[0]))
    return list(c.values())

### Number of Sentences

In [13]:
def NumberOfSentences(tweet):
    sentences = sent_tokenize(tweet)
    return len(sentences)

### Number of Words
using nltk's TweetTokenizer

In [14]:
def NumberOfWords(tweet):
    tknzr = TweetTokenizer()
#     words = word_tokenize(tweet)
    tweet_words = tknzr.tokenize(tweet)
    return len(tweet_words), tweet_words

### Readability Score
using Flesch–Kincaid readability tests (higher score means the tweet is more easy to read)

In [15]:
def Readability(tweet):
    totalSentences = float(NumberOfSentences(tweet))
    totalTweetWords, tweetWords = NumberOfWords(tweet)
    totalTweetWords = float(totalTweetWords)
    regexpURL = re.compile(r'https?:\/\/.[^\s]*|www\.[^\s]*')
    totalSyllables = 0.0
    for word in tweetWords:
        if not regexpURL.search(word):
            hyphenated = PYPHEN_DIC.inserted(word)
            syllables = hyphenated.count("-") + 1 - hyphenated.count("--")
            totalSyllables += syllables
#         else:
#             totalTweetWords -= 1.0
    if totalSentences > 0 and totalTweetWords > 0:
        score = 206.835 - 1.015 * (totalTweetWords/totalSentences) - 84.6 * (totalSyllables/totalTweetWords)
    else:
        print("Readability issue")
        score = 0.0
    return score

### Feature Names

In [16]:
fNames = ['nos', 'now', 'readabilityScore', 'noclPerWord', 'noe', 'nocf', 'noURL', 'maxWordAppearancePerTweet', 'nowr']
fNames.extend(all_pos_tags)
fNames.extend(topicNames)
fNames = ['avg_'+f for f in fNames]
fNames.append('maxWordAppearancePerProfile')
fNames = sorted(fNames)
print(len(fNames))
print(fNames)
fNames_dir = 'C:/Users/Vanda/Documents/PAN19/data/pan19-author-profiling-training-2019-01-28/en_features/'
if not os.path.exists(fNames_dir):
    os.makedirs(fNames_dir)
pickle.dump(fNames, open(fNames_dir+'feature_names.p', 'wb'))

164
['avg_ADJ', 'avg_ADP', 'avg_ADV', 'avg_AUX', 'avg_CCONJ', 'avg_CONJ', 'avg_DET', 'avg_EOL', 'avg_INTJ', 'avg_NOUN', 'avg_NO_TAG', 'avg_NUM', 'avg_PART', 'avg_PRON', 'avg_PROPN', 'avg_PUNCT', 'avg_SCONJ', 'avg_SPACE', 'avg_SYM', 'avg_Tops', 'avg_VERB', 'avg_X', 'avg_act', 'avg_adjectives_for_people', 'avg_animal', 'avg_april_fool', 'avg_art', 'avg_artifact', 'avg_astronomy', 'avg_attribute', 'avg_baseball', 'avg_bathroom', 'avg_beach', 'avg_big', 'avg_biomes', 'avg_birds', 'avg_birthday', 'avg_boat', 'avg_bodies_of_water', 'avg_body', 'avg_buildings', 'avg_camping', 'avg_car', 'avg_carnival', 'avg_carparts', 'avg_castle', 'avg_cats', 'avg_change', 'avg_christmas', 'avg_circus', 'avg_clothes', 'avg_cognition', 'avg_colors', 'avg_communication', 'avg_competition', 'avg_computer', 'avg_constitution', 'avg_consumption', 'avg_contact', 'avg_container', 'avg_cooking', 'avg_cooking_tools', 'avg_country', 'avg_creation', 'avg_dance', 'avg_dentist', 'avg_desserts', 'avg_doctor', 'avg_dogs', 

### Compute features for each tweet
ngrams are not used

In [17]:
def GetFeatures(tweet):
    features = {}
    features['nos'] = NumberOfSentences(tweet)
    features['now'], words = NumberOfWords(tweet)
    features['readabilityScore'] = Readability(tweet)
    nocl, features['noclPerWord'] = NumberOfCapitalLetters(tweet)
    features['noe'] = NumberOfEmoticons(tweet)
    features['nocf'] = CharacterFlooding(tweet)
    features['noURL'] = NumberOfURLs(tweet)
    features['maxWordAppearancePerTweet'], features['nowr'] = wordsRepeated(tweet)
#     ngram = nGram(tweet, ngram_model)
    pos = POSTags(tweet)
    posTags = sorted(all_pos_tags)
    for i in range(len(posTags)):
        features[posTags[i]] = pos[i]
    topicValues = Topics(tweet)
    for i in range(len(topicNames)):
        features[topicNames[i]] = topicValues[i]
#     features = [nos, now, readabilityScore, avg_nocl, noe, nocf]
#     ngramFeatures = nGram(tweet, ngram_model)
#     for i in range(ngramFeatures.shape[1]):
#         name = ngram_features[i]
#         feat = ngramFeatures[0,i]
    return features

### Max number of word repetitions for a profile

In [18]:
def maxWordRepetitionsPerProfile(TweetGenerator):
    tknzr = TweetTokenizer()
    frequency = defaultdict(int)
    for tweet in TweetGenerator:
        tweet = tweet['data']
        tweet = spacy_nlp(tweet)
        for token in tweet:
            if token.lemma_ not in spacy_stopwords:
                frequency[token.lemma_] += 1
    maxAppearance = 0
    if len(frequency.items()) > 0:
        maxAppearance = max(frequency.items(), key=operator.itemgetter(1))[1]
    return maxAppearance

In [19]:
def additionalProfileFeatures(TweetGenerator, features, featureNames):
    maxWordAppearance = maxWordRepetitionsPerProfile(TweetGenerator)
    features.append(maxWordAppearance)
    featureNames.append('maxWordAppearancePerProfile')
    return features, featureNames

### Compute average of features for each Twitter profile

In [20]:
def cummulateFeatures(profileFeatures):
    denom = len(profileFeatures)
    cummulatedFeatures = defaultdict(float)
    for tweetFeature in profileFeatures:
        for featureName in tweetFeature.keys():
            cummulatedFeatures[('avg_'+featureName)] += tweetFeature[featureName]
    for k, v in cummulatedFeatures.items():
        cummulatedFeatures[k] = cummulatedFeatures[k]/denom
    cummulatedFeatures = OrderedDict(sorted(cummulatedFeatures.items(), key=lambda e: e[0]))
    featureNames = list(cummulatedFeatures.keys())
    features = list(cummulatedFeatures.values())
    return features, featureNames

In [21]:
def writeProfileFeatures(directory, profileID, profileFeatures, Truth, TweetGenerator):
    profileName = Truth[0][profileID]
#     mode = roleDict[profileName]
    features, featureNames = cummulateFeatures(profileFeatures)
    features, featureNames = additionalProfileFeatures(TweetGenerator,  features, featureNames)
    features = [str(f) for f in features]
    features = '\t'.join(features)
    with codecs.open(directory+'/features.txt', "a", "utf-8-sig") as text_file:
        text_file.write(profileName+'\t'+features+'\t'+Truth[1][profileID]+'\t'+Truth[2][profileID]+'\n')

In [22]:
def LoadProfile(input_file):    
    Profile = ET.parse(input_file)
    ProfileRoot = Profile.getroot()
    Profile_attr = ProfileRoot.attrib
    for tweet in Profile.iter('document'):        
        tweet_dict = Profile_attr.copy()
        tweet_dict.update(tweet.attrib)
        tweet_dict['data'] = tweet.text
        yield tweet_dict

In [23]:
def ProcessFolder(root_directory, input_directory, output_directory):    
    #Create output directory (if it does not exist yet)
    profile_directory = root_directory+'/'+output_directory+'/'+'profile'
    tweet_directory = root_directory+'/'+output_directory+'/'+'tweet'
    if os.path.exists(root_directory+'/'+output_directory):
        shutil.rmtree(root_directory+'/'+output_directory)
    os.mkdir(root_directory+'/'+output_directory)
    os.mkdir(profile_directory)
    os.mkdir(tweet_directory)
    #Read labels (and file names)
    Truth = pd.read_csv(root_directory+'/'+input_directory+'/truth.txt', sep=":::", header=None, engine='python')
    #Iterate over all user names, and process the corresponding file names
    for i in range(0,Truth.shape[0]):
#     for i in range(0,2):        
        #Open text file for output   
        print(Truth[0][i])
        profileFeatures = []
        TweetGenerator = LoadProfile(root_directory+'/'+input_directory+'/'+Truth[0][i]+'.xml')
        with codecs.open(tweet_directory+'/'+Truth[0][i]+'.txt', "w", "utf-8-sig") as text_file: 
            for tweet in TweetGenerator:
                tweetFeatures = GetFeatures(tweet['data'])
                profileFeatures.append(tweetFeatures)
                tweetFeatures = [str(f) for f in tweetFeatures.values()]
                tweetFeatures = '\t'.join(tweetFeatures)
                text_file.write(tweetFeatures + '\n')   
        TweetGenerator = LoadProfile(root_directory+'/'+input_directory+'/'+Truth[0][i]+'.xml')
        writeProfileFeatures(profile_directory, i, profileFeatures, Truth, TweetGenerator)

In [ ]:
ProcessFolder('C:\\Users\\Vanda\\Documents\\PAN19\\data\\pan19-author-profiling-training-2019-01-28', 'en', 'en_features')

b2d5748083d6fdffec6c2d68d4d4442d
2bed15d46872169dc7deaf8d2b43a56
25395a7dbd2caa3d828bb3dbd57d8857
1c9f161414334b286c4dc70163744390
1da1f87b3dc778f28268eec70ce94f19
bd4286bfafb8a35b8e132a396b884e07
af61c4c017f246da69285497baf3dc0b
7a36090b20e7bddbe55561c52f959041
5183ff5bedcab9a6a301ff04e27166cd
6ac10734f35a773e9f2209f8668fffdf
b5b06752b42d3c354cc8e126f030a864
e2334dbd4f17861a5305e808eedbd195
c6cb83581e942ea1b07a607c5eaefb49
6270e73e5cf9a83ec96faa7d3972f9e1
991a426dde6803bf77b6477a2c686ef2
fed0a94b487a37d43313c2e10841d11f
344fc6cfa2762bebdf7d49afb56e2ee7
4f29c1ea09b2d6ef919b31d86e243646
48ea8296dc261f8c744ceb054b089ca3
707549dcdb6037769bc87b51821ca9eb
7473a712f9d25003c0b262da77d05a81
60840ddc48c986a1e3c77d2416a146b4
ff339848f33aeeefacea8248d36604d2
71f7d13771bc9b2662c759bdb0214c0f
8e97035efae463eea9b2b8ddc1a55f51
a16d865fe24049e64d497346e55d9f9e
10d5df00273af4b02509ca9813224ae5
953aac410bf1c9f8b14dc00c07d5e8eb
37c988debca526d34a2f00026d8243c1
66cc6969c3b446e9efbcf30e6d1a70d9
d296fc545f5

8db1004ec802f88e02a5a93efa1682a5
5c54d581bed9be4b03440a1d6523d19
68672356fae13b2c2a7bb690c1c7f672
2db817b54c37727524a6d9187781757c
7c8739c9fc9b7a47f6f956ae02592297
6042abb6ae2897101443b77da1bf4d18
269fe3bd22b0ac252c3557338edba9a2
7c64ac28a856b356a525ba9ebd395995
724f2b60af0bc2f1332b40c53f7a1b35
d6573c4552109975202abd85659307ab
64673a899b73c574e589ef0fe3f7d421
a3fbd41fe502077446a98c2dfcc78987
f130569a3b63b7d5590159afb8bb8bdb
27c105b79821c6183c9d7a7eff84d548
2ead2be5e2a767ff12e931475759945e
9dc76a3fc5f0edbed4fa2f26c712a932
d4f4966de7c8872e0397601c758949de
f3c1b8db693df2b5c3de19c2ec941582
a4e7e1d994388da33994796c125dc1d7
bf0975cb529ac583e11535541001ce9d
8f2c2cf696f60fb8ec4747771d4f0fc7
5436789952b801369e975b6b5552cda
ad24ff15414362baaf498f58ac51457
be75dd88e8e13182ea5029f93709dcc7
597284527e084d7d95d62d665291fcd9
da8c3f13cc66a4be198be4917981955b
4cf45c99cfa92c181b55e7474a63af8f
d37b5f26396857be1c19b988b7652147
7183fcc7245a56a10ad4c362d06ac131
ca0cfcc9f775d6a41c3bab0b9fbbec79
9698310fc6312

21a55d22eda995f36712d12012e17500
f91d065f8506c5b30214e8a3e6462433
19d21464af3fe4d742d32e5afa8b2ff8
1bb1d130aab6bd9c04a441f187ed0dc0
a15213ff90b871d847f330b593ff1da4
a9e06d3a027b502848c68051b0f0973d
e3bc109046e8f8e9d2765603f79da3d5
a208e82ec083f32d3ba479fd698a7978
1728fc13e288b4cb103f254241cf239b
6bcfb423938bc6d8b567c656a4287ab7
cf0e3b50c1cf15e399186f911f66bcfd
58b9737963b35c508f11180a8cec6ae1
3c5c3a1ee3989406b38ea9e849da1f5
d77f6b0feca3c25493bab69a6078d88c
bdf170fdf5f85577182c8212d2324625
581869430eacca04a6a2024d3a03d2b2
9f91a99a0aa9453b82dfcb86e290577d
a20c87c2d2ccf981577dc645592b9629
8380028f752134a5fee5cae7670ba2dc
23d2238fe67831e61fa5a46f908efc9e
9835fa3492b40c8590af134b67b09fa2
7893589279f5f40196c6121cd6895684
3333347d17493268e34838c907b1e102
f257093743747b0c86d5e5893b5989c
c5abeb7ee96cfdf2d99261afe02f9057
a20e37e1de4906e29724d37c22684978
3c06fc313a1c3bdd3d420590fb88c145
6fc6b9da3bc9d15ee76c6b895ab433b3
6b2a975d07cbab5a1367aa8869afc90
68a906cb9146767976bb2db6ea04322d
5cf7435b1f72b